We're going to rolls thousands of dice dozens of times until all our lockpicks break or the locks are opened.  Since we don't want to do this one dice at a time we can take advantage of the parallel processing power of the numpy python package which can generate and compare millions of random numbers in seconds.

In [48]:
# Import numpy, because numpy is the best part of python.
# If you disagree fight me.
import numpy as np

# I'm going to do some comparisons asking whether an object caled NaN (not a number) is greater than some number.
# That's going to cause the computer to give a warning telling me that math doesn't work that way.
# I don't care about that warning so I'm preemptively turning them off.
np.warnings.filterwarnings('ignore')

In [92]:
# I'm writing this as a function because that's generally good practice.
def PickLocks(DC, Bonus, SuccessesNeeded, NumRuns):
    #######################################################################
    # This will keep track of NumRuns worth of attempts to pick the lock,
    # as picks break or the locks break we set those attempts to NaN so no
    # more math can be done on them.
    #
    # DC -- An integer giving the difficulty class
    # Bonus -- An integer giving the bonus to the roll
    # SuccessesNeeded -- An integer giving the number of successes needed to
    #                    pick the lock successfully.
    # NumRuns - An integer giving the number of simulations to run.
    #######################################################################
    
    # We will keep track of the number of successes each lock has on it in this array
    # This creates a big list that looks like [0, 0, 0, ...]
    Progress = np.zeros(NumRuns)
    
    # Keep track of the number of locks we have picked, starting at 0.
    Picked = 0

    # This will loop as long as the condition is true, here we are saying keep running
    # until every simulation is set to NaN, which happens on both picks breaking or the
    # lock being successfully opened.
    while np.sum(np.isnan(Progress)) < NumRuns:
        
        # Generate a random roll for each simulation, the 20 tells it to use random integers
        # from 0 to 19, so we add 1 to it then add the bonus.  The astype('float') is there
        # because numpy refuses to mix NaN into an array of integers, so we convert the rolls
        # to 0.0, 1.0, etc.
        Rolls = np.random.randint(20, size = NumRuns).astype('float') + 1 + Bonus

        # find all the picks that broke and take those simulations out of the running by setting
        # them to NaN.  There are two ways to crit fail so we check both: getting 10 below the DC
        # or (|) we rolled a 1 and (&) failed to hit the DC.
        Progress[np.where( (Rolls <= DC-10) | ( (Rolls == 1 + Bonus) & (Rolls < DC) ) ) ] = np.NaN

        # Check for critical successes, we only add 1 because the success check will add the other
        # one we need.This checks two conditions: is the total DC + 10 or we both rolled a 20 and
        # the roll meets the DC.
        Progress[np.where( (Rolls >= DC+10 ) | ( (Rolls == 20 + Bonus) & (20 + Bonus >= DC) ))] += 1
        
        # Check for regular success.  This also checks to two conditions, one that we met the DC and
        # the other is that we roll a 20 and didn't crit fail.
        Progress[np.where(( (Rolls > DC) | ( (Rolls == 20 + Bonus) & (20 + Bonus > DC - 10) )))] += 1

        # Look for any attempts that have met all the successes needed and count them.  The [0] at the
        # end is needed because of how numpy formats the where function, without it we get the length +1
        # since numpy also stores the datatype in the array as an object.  The [0] tells it to return
        # item 0 (the array of all the successful attempts only).
        Picked += len(np.where(Progress >= SuccessesNeeded)[0])
        
        # With that done we end the runs that ended in a picked lock.
        Progress[np.where(Progress >= SuccessesNeeded)] = np.NaN
    
    # Now we return the number of picked locks.  So when you call the PickLocks function all you get back
    # is an integer giving you the number of locks that got picked.
    return Picked

In [116]:
# With the function written we set our parameters
DC = 24
Bonus = 10
SuccessesNeeded = 3
# The larger this number the more accurate your simulation will be.  100,000 should be plenty.
NumRuns = 100000

In [117]:
# The number of runs that ended in a picked lock divided by the number of runs we tried is what we want.
Percent = 100 * PickLocks(DC, Bonus, SuccessesNeeded, NumRuns)/NumRuns

In [118]:
# Now we print it, we could just call print(Percent) or even just type Percent, but the format will be ugly.
# So we use an f string, telling it to print a sentence and fill in the variable Percent in there, truncating
# the variable at 2 decimal places (the :.2f part).
print(f'{Percent:.2f}% of runs ended in a picked lock')

26.17% of runs ended in a picked lock
